In [ ]:
import numpy as np # load point clouds and more array/vector operations
import cv2 # load images
import plotly.graph_objects as go # visualize point clouds

In [ ]:
root = "TASK1/dataset/Record1/"
np.version.version


In [ ]:
def inv_rot(rot):
    """
        Calculates for a given 4x4 transformation matrix (R|t) the inverse.
    """
    inv_rot_mat = np.zeros((4,4))
    inv_rot_mat[0:3,0:3] = rot[0:3,0:3].T
    inv_rot_mat[0:3,3] = -np.dot(rot[0:3,0:3].T, rot[0:3,3])
    inv_rot_mat[3,3] = 1
    return inv_rot_mat

In [ ]:
def transfer_points(points, rot_t):
    """
        Calculates the transformation of a point cloud for a given transformation matrix.
    """
    points = np.concatenate([points, np.ones([1,points.shape[1]])])
    points[0:3,:] = np.dot(rot_t, points[0:4,:])[0:3, :]
    return points[0:3]

In [ ]:
def make_boundingbox(label):
    """
        Calculates the Corners of a bounding box from the parameters.
    """
    corner = np.array([
        [+ label[3]/2, + label[4]/2, + label[5]/2],
        [+ label[3]/2, + label[4]/2, - label[5]/2],
        [+ label[3]/2, - label[4]/2, + label[5]/2],
        [+ label[3]/2, - label[4]/2, - label[5]/2],
        [- label[3]/2, + label[4]/2, + label[5]/2],
        [- label[3]/2, - label[4]/2, + label[5]/2],
        [- label[3]/2, + label[4]/2, - label[5]/2],
        [- label[3]/2, - label[4]/2, - label[5]/2],
    ])
    print(label[0])
    print(label[1])
    print(label[2])
    corner = transfer_points(corner.T, rt_matrix(yaw = label[6])).T
    corner = corner + label[0:3]
    return corner

In [ ]:
def rt_matrix(x=0, y=0, z=0, roll=0, pitch=0, yaw=0):
    """
        Calculates a 4x4 Transformation Matrix. Angels in radian!
    """
    c_y = np.cos(yaw)
    s_y = np.sin(yaw)
    c_r = np.cos(roll)
    s_r = np.sin(roll)
    c_p = np.cos(pitch)
    s_p = np.sin(pitch)
    
    rot = np.dot(np.dot(np.array([[c_y,-s_y,0],
                                     [s_y,c_y,0],
                                     [0,0,1]]),
                           np.array([[c_p,0,s_p],
                                     [0,1,0],
                                     [-s_p,0,c_p]])),
                            np.array([[1,0,0],
                                     [0,c_r,-s_r],
                                     [0,s_r,c_r]]))
    matrix = np.array([[0,0,0,x],
                     [0,0,0,y],
                     [0,0,0,z],
                     [0,0,0,1.]])
    matrix[0:3,0:3] += rot
    return matrix

In [ ]:
rotationmat = np.load(root + "calibrationMat.npy")
invrotationmat = inv_rot(rotationmat)

In [ ]:
ind = 20
blick = np.loadtxt(root + "Blickfeld/point_cloud/%06d.csv" % ind)
velo = np.loadtxt(root + "Velodyne/point_cloud/%06d.csv" % ind)
velo = transfer_points(velo.T[0:3], rotationmat).T

m=len(blick)
print("total number of points in blickfield = ",m)

v=len(velo)
print("total number of points in velo = ",v)

bb = np.loadtxt(root + "/Blickfeld/bounding_box/%06d.csv" % ind)
"before transferring bb into make_bounding box it represents bounding box internal values such as x,y,z,w,h etc"
"So we will take the middle point and calculate the distance from the centre point to LIDAR"
dist=((bb[0]**2)+(bb[1]**2)+(bb[2]**2))**(0.5)
print("distance = ",dist)

bb = make_boundingbox(bb)
print(np.array(bb))
bbmaxarray=bb.max(axis=0)
bbminarray=bb.min(axis=0)
print(bbmaxarray[0])
print(bbmaxarray[1])
print(bbmaxarray[2])
print(bbminarray[0])
print(bbminarray[1])
print(bbminarray[2])
"now we have two arrays of max and min value(corner points)"

"We will do the same stuff for Velodyne data"

bbv = np.loadtxt(root + "/Velodyne/bounding_box/%06d.csv" % ind)
"before transferring bb into make_bounding box it represents bounding box internal values such as x,y,z,w,h etc"
"So we will take the middle point and calculate the distance from the centre point to LIDAR"
distv=((bbv[0]**2)+(bbv[1]**2)+(bbv[2]**2))**(0.5)
print("distance = ",distv)

bbv = make_boundingbox(bbv)
print(np.array(bbv))
bbmaxarrayv=bbv.max(axis=0)
bbminarrayv=bbv.min(axis=0)
print(bbmaxarrayv[0])
print(bbmaxarrayv[1])
print(bbmaxarrayv[2])
print(bbminarrayv[0])
print(bbminarrayv[1])
print(bbminarrayv[2])
"now we have two arrays of max and min value(corner points)"



In [ ]:
blick
print(blick[0][1])
print(blick[1][0])
print(blick[2][2])

n = 0 
count = 0 
while n != m:
    if bbmaxarray[0] >= blick[n][0] >= bbminarray[0] and bbmaxarray[1] >= blick[n][1] >= bbminarray[1] and bbmaxarray[2] >= blick[n][2] >= bbminarray[2]:
       count += 1 
    
    n += 1 

print("Number of times loop iterated for blickfield = ",n)
print("Resolution(blickfield) =  ",count,"with Index = ",ind,"at a distance of(in m)= ",dist)

"Distance calculation we require x,y and z coordinate,Which we will get from label[0], label[1] and label[2] respectively"


velo
print(velo[0][1])
print(velo[1][0])
print(velo[2][2])

nv = 0 
countv = 0 
print("velodyne point cloud numbers = ",v)
while nv != v:
    if bbmaxarrayv[0] >= velo[nv][0] >= bbminarrayv[0] and bbmaxarrayv[1] >= velo[nv][1] >= bbminarrayv[1] and bbmaxarrayv[2] >= velo[nv][2] >= bbminarrayv[2]:
       countv += 1 
    
    nv += 1 

print("Number of times loop iterated for Velodyne = ",nv)
print("Resolution(Velodyne) =  ",countv,"with Index = ",ind,"at a distance of(in m)= ",distv)

"Distance calculation we require x,y and z coordinate,Which we will get from label[0], label[1] and label[2] respectively"




In [ ]:
data = [go.Scatter3d(x = blick[:,0],
                     y = blick[:,1],
                     z = blick[:,2],
                    mode='markers', type='scatter3d',
                    text=np.arange(blick.shape[0]),
                    marker={
                        'size': 2,
                        'color': "blue",
                        'colorscale':'rainbow',
}),
        go.Scatter3d(x = velo[:,0],
                     y = velo[:,1],
                     z = velo[:,2],
                    mode='markers', type='scatter3d',
                    text=np.arange(velo.shape[0]),
                    marker={
                        'size': 2,
                        'color': "green",
                        'colorscale':'rainbow',
}),
        go.Scatter3d(x = bb[:,0],
                     y = bb[:,1],
                     z = bb[:,2],
                     text=np.arange(8),
                    mode='markers', type='scatter3d',
                    marker={
                        'size': 2,
                        'color': "red",
                        'colorscale':'rainbow',
})
]
layout = go.Layout(
    scene={
        'xaxis': {'range': [-25, 25], 'rangemode': 'tozero', 'tick0': -5},
        'yaxis': {'range': [-25, 25], 'rangemode': 'tozero', 'tick0': -5},
        'zaxis': {'range': [-12.5, 12.5], 'rangemode': 'tozero'}
    }
)
go.Figure(data=data, layout = layout)

In [ ]:
ind = 17
blick = np.loadtxt(root + "Blickfeld/point_cloud/%06d.csv" % ind)
velo = np.loadtxt(root + "Velodyne/point_cloud/%06d.csv" % ind)
blick = transfer_points(blick.T[0:3], invrotationmat).T

bb = np.loadtxt(root + "/Velodyne/bounding_box/%06d.csv" % ind)
bb = make_boundingbox(bb)

In [ ]:
data = [go.Scatter3d(x = blick[:,0],
                     y = blick[:,1],
                     z = blick[:,2],
                    mode='markers', type='scatter3d',
                    text=np.arange(blick.shape[0]),
                    marker={
                        'size': 2,
                        'color': "blue",
                        'colorscale':'rainbow',
}),
        go.Scatter3d(x = velo[:,0],
                     y = velo[:,1],
                     z = velo[:,2],
                    mode='markers', type='scatter3d',
                    text=np.arange(velo.shape[0]),
                    marker={
                        'size': 2,
                        'color': "green",
                        'colorscale':'rainbow',
}),
        go.Scatter3d(x = bb[:,0],
                     y = bb[:,1],
                     z = bb[:,2],
                     text=np.arange(8),
                    mode='markers', type='scatter3d',
                    marker={
                        'size': 2,
                        'color': "red",
                        'colorscale':'rainbow',
})
]
layout = go.Layout(
    scene={
        'xaxis': {'range': [-25, 25], 'rangemode': 'tozero', 'tick0': -5},
        'yaxis': {'range': [-25, 25], 'rangemode': 'tozero', 'tick0': -5},
        'zaxis': {'range': [-25, 25], 'rangemode': 'tozero'}
    }
)
go.Figure(data=data, layout = layout)

In [ ]:
root = "TASK1/dataset/Record2/"

In [ ]:
rotationmat = np.load(root + "calibrationMat.npy")
invrotationmat = inv_rot(rotationmat)

In [ ]:
ind = 9
blick = np.loadtxt(root + "Blickfeld/point_cloud/%06d.csv" % ind,)
velo = np.loadtxt(root + "Velodyne/point_cloud/%06d.csv" % ind)
velo = transfer_points(velo.T[0:3], rotationmat).T

bb = np.loadtxt(root + "/Blickfeld/bounding_box/%06d.csv" % ind)
if len(bb):
    bb = make_boundingbox(bb)
else:
    bb = np.array([[None,None,None,None,None,None,None]])

In [ ]:
data = [go.Scatter3d(x = blick[:,0],
                     y = blick[:,1],
                     z = blick[:,2],
                    mode='markers', type='scatter3d',
                    text=np.arange(blick.shape[0]),
                    marker={
                        'size': 2,
                        'color': "blue",
                        'colorscale':'rainbow',
}),
        go.Scatter3d(x = velo[:,0],
                     y = velo[:,1],
                     z = velo[:,2],
                    mode='markers', type='scatter3d',
                    text=np.arange(velo.shape[0]),
                    marker={
                        'size': 2,
                        'color': "green",
                        'colorscale':'rainbow',
}),
        go.Scatter3d(x = bb[:,0],
                     y = bb[:,1],
                     z = bb[:,2],
                     text=np.arange(8),
                    mode='markers', type='scatter3d',
                    marker={
                        'size': 2,
                        'color': "red",
                        'colorscale':'rainbow',
})
]
layout = go.Layout(
    scene={
        'xaxis': {'range': [-45, 45], 'rangemode': 'tozero', 'tick0': -5},
        'yaxis': {'range': [-45, 45], 'rangemode': 'tozero', 'tick0': -5},
        'zaxis': {'range': [-12.5, 12.5], 'rangemode': 'tozero'}
    }
)
go.Figure(data=data, layout = layout)

In [ ]:
ind = 8
blick = np.loadtxt(root + "Blickfeld/point_cloud/%06d.csv" % ind,)
velo = np.loadtxt(root + "Velodyne/point_cloud/%06d.csv" % ind)
blick = transfer_points(blick.T[0:3], invrotationmat).T

bb = np.loadtxt(root + "/Velodyne/bounding_box/%06d.csv" % ind)
if len(bb):
    bb = make_boundingbox(bb)
else:
    bb = np.array([[None,None,None,None,None,None,None]])

In [ ]:
data = [go.Scatter3d(x = blick[:,0],
                     y = blick[:,1],
                     z = blick[:,2],
                    mode='markers', type='scatter3d',
                    text=np.arange(blick.shape[0]),
                    marker={
                        'size': 2,
                        'color': "blue",
                        'colorscale':'rainbow',
}),
        go.Scatter3d(x = velo[:,0],
                     y = velo[:,1],
                     z = velo[:,2],
                    mode='markers', type='scatter3d',
                    text=np.arange(velo.shape[0]),
                    marker={
                        'size': 2,
                        'color': "green",
                        'colorscale':'rainbow',
}),
        go.Scatter3d(x = bb[:,0],
                     y = bb[:,1],
                     z = bb[:,2],
                     text=np.arange(8),
                    mode='markers', type='scatter3d',
                    marker={
                        'size': 2,
                        'color': "red",
                        'colorscale':'rainbow',
})
]
layout = go.Layout(
    scene={
        'xaxis': {'range': [-50, 50], 'rangemode': 'tozero', 'tick0': -5},
        'yaxis': {'range': [-50, 50], 'rangemode': 'tozero', 'tick0': -5},
        'zaxis': {'range': [-50, 50], 'rangemode': 'tozero'}
    }
)
go.Figure(data=data, layout = layout)